## This is the Capstone Project Notebook.

In [1]:
import pandas as pd
import numpy as np
#import sys
#!conda install --yes --prefix {sys.prefix} lxml
!conda install -c anaconda --yes lxml 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         156 KB  anaconda

The following packages will be UPDATED:

    certifi: 2019.11.28-py36_0 conda-forge --> 2019.11.28-py36_0 anaconda
    openssl: 1.1.1d-h516909a_0 conda-forge --> 1.1.1-h7b6447c_0  anaconda


certifi-2019.11.28   | 156 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [2]:
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge

The following packages will be UPDATED:

    certifi: 2019.11.28-py36_0 anaconda --> 2019.11.28-py36_0 conda-forge

The following packages will be DOWNGRADED:

    openssl: 1.1.1-h7b6447c_0  anaconda --> 1.1.1d-h516909a_0 conda-forge


certifi-2019.11.28   | 149 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
import requests # library to handle requests

In [4]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [5]:
!conda install -c conda-forge geocoder --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [88]:
from geopy.geocoders import Nominatim
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors

In [26]:

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url, header=0)[0]

df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


#### Drop rows that a borough is Not assigned.

In [27]:
index_empty=df[df['Borough']=='Not assigned'].index
df.drop(index_empty,inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Not assigned
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [28]:
df['Neighbourhood'] = df.apply(
    lambda row: row['Borough'] if row['Neighbourhood']=='Not assigned' else row['Neighbourhood'],
    axis=1
)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


#### Merge postal code area

In [29]:
df_post=df.groupby(['Postcode'])['Neighbourhood'].apply(lambda x: ' ,'.join(x)) 

In [30]:
df_post=pd.DataFrame(data=df_post)
df_post.reset_index(inplace=True)
df_post.head()

,Postcode,Neighbourhood
0,M1B,"Rouge ,Malvern"
1,M1C,"Highland Creek ,Rouge Hill ,Port Union"
2,M1E,"Guildwood ,Morningside ,West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


In [31]:
df1=df[['Postcode','Borough']].drop_duplicates()
df1.head()

,Postcode,Borough
0,M3A,North York
1,M4A,North York
2,M5A,Downtown Toronto
3,M6A,North York
5,M7A,Downtown Toronto


In [32]:
df2= df1.merge(df_post, on='Postcode')
df2.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights ,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [33]:
df2.shape

(103, 3)

There are 103 rows in the dataset.

In [34]:
import geocoder # import geocoder

In [35]:
# initialize your variable to None
def get_postcode(postal_code):
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return [latitude,longitude]

In [36]:
postal_code='M3A'
geocoder.google('{}, Toronto, Ontario'.format(postal_code))

<[REQUEST_DENIED] Google - Geocode [empty]>

In [37]:
geourl='https://cocl.us/Geospatial_data'
df_geo=pd.read_csv(geourl)

In [38]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merge the two dataframes.

In [49]:
df_geo.rename({'Postal Code': 'Postcode'},axis=1,inplace=True)
df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [50]:
df3= pd.merge(df2, df_geo, on='Postcode')

In [51]:
df3.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights ,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


#### Define Foursquare Credentials and Version

In [52]:
CLIENT_ID = 'G24RTX4QGMM2GUGHEVQHFJKKIUZC54MR0L2Q5PVWTU1FNEZL' # your Foursquare ID
CLIENT_SECRET = '0YBQEOWRXWSHMZ0OSGV0ZPEGHW1AO5IZ5YRGK3JTB3KOA5KQ' # your Foursquare Secret
#CLIENT_ID = 'your-client-ID' # your Foursquare ID
#CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100
radius=500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G24RTX4QGMM2GUGHEVQHFJKKIUZC54MR0L2Q5PVWTU1FNEZL
CLIENT_SECRET:0YBQEOWRXWSHMZ0OSGV0ZPEGHW1AO5IZ5YRGK3JTB3KOA5KQ


#### Request data from Foursquare for one postcode to see how data looks like

In [53]:
lat=df3.iloc[0]['Latitude']
lng=df3.iloc[0]['Longitude']
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

In [54]:
results = requests.get(url).json()["response"]['groups'][0]['items']
results

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
   'name': 'Brookbanks Park',
   'location': {'address': 'Toronto',
    'lat': 43.751976046055574,
    'lng': -79.33214044722958,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.751976046055574,
      'lng': -79.33214044722958}],
    'distance': 245,
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['Toronto', 'Toronto ON', 'Canada']},
   'categories': [{'id': '4bf58dd8d48988d163941735',
     'name': 'Park',
     'pluralName': 'Parks',
     'shortName': 'Park',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-4e8d9dcdd5fbbbb6b3003c7b-0'},
 {'reasons': {'count': 0,
   'items': [{'

#### Only keep Boroughs in Toronto.

In [55]:
toronto_data = df3[df3['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()
toronto_data['Borough'].value_counts()


Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

#### Define the function to extract Fouresquare data

In [56]:
def getNearbyVenues(postcodes, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for postcode, lat, lng in zip(postcodes, latitudes, longitudes):
        print(postcode)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postcode, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [57]:
toronto_venues = getNearbyVenues(postcodes=toronto_data['Postcode'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


M5A
M7A
M5B
M5C
M4E
M5E
M5G
M6G
M5H
M6H
M5J
M6J
M4K
M5K
M6K
M4L
M5L
M4M
M4N
M5N
M4P
M5P
M6P
M4R
M5R
M6R
M4S
M5S
M6S
M4T
M5T
M4V
M5V
M4W
M5W
M4X
M5X
M4Y
M7Y


In [58]:
toronto_venues.head(5)

,Postcode,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,M5A,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [64]:
toronto_venues.columns

Index(['Postcode', 'Latitude', 'Longitude', 'Venue', 'Venue Latitude',
       'Venue Longitude', 'Venue Category'],
      dtype='object')

#### How many unique types of venues?

In [65]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 229 uniques categories.


#### Analyze Neighborhoods

Process data

In [66]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postcode'] = toronto_venues['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postcode,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Frequency of each venue type

In [68]:
toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
toronto_grouped.head()

,Postcode,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.20000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000
1,M4K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023810,0.0,...,0.02381,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.02381
2,M4L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000
3,M4M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.02381,0.0,0.0,0.02381
4,M4N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000


In [69]:
toronto_grouped.shape

(39, 230)

##### The 5 most common types of venues in each neighborhood

In [71]:
num_top_venues = 5

for postcode in toronto_grouped['Postcode']:
    print("----"+postcode+"----")
    temp = toronto_grouped[toronto_grouped['Postcode'] == postcode].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
               venue  freq
0  Health Food Store   0.2
1                Pub   0.2
2               Park   0.2
3       Neighborhood   0.2
4              Trail   0.2


----M4K----
                    venue  freq
0        Greek Restaurant  0.21
1             Coffee Shop  0.10
2      Italian Restaurant  0.07
3          Ice Cream Shop  0.05
4  Furniture / Home Store  0.05


----M4L----
                  venue  freq
0         Burrito Place  0.06
1  Fast Food Restaurant  0.06
2        Sandwich Place  0.06
3            Steakhouse  0.06
4          Burger Joint  0.06


----M4M----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.07
2   Italian Restaurant  0.05
3  American Restaurant  0.05
4            Gastropub  0.05


----M4N----
                venue  freq
0                Park   0.2
1                Lake   0.2
2         Swim School   0.2
3  Dim Sum Restaurant   0.2
4            Bus Line   0.2


----M4P----
               venue  freq
0            Do

#### Cluster Neighborhoods

Put the most common venues to a dataframe

In [72]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [78]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = toronto_grouped['Postcode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Pub,Park,Neighborhood,Health Food Store,Trail,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
1,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Yoga Studio,Bubble Tea Shop,Sports Bar,Spa,Juice Bar
2,M4L,Pet Store,Pub,Burger Joint,Burrito Place,Liquor Store,Sandwich Place,Fast Food Restaurant,Italian Restaurant,Intersection,Fish & Chips Shop
3,M4M,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Italian Restaurant,Gastropub,Yoga Studio,Fish Market,Pet Store
4,M4N,Lake,Park,Swim School,Bus Line,Dim Sum Restaurant,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant


Now cluster them based on most common venues.

In [79]:
from sklearn.cluster import KMeans
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 2, 2, 0, 2, 2, 2, 4, 2], dtype=int32)

Add labels to neighborhoods

In [80]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Postcode'), on='Postcode')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,2,Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Breakfast Spot,Café,Yoga Studio,Farmers Market,Restaurant
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,2,Coffee Shop,Park,Gym,Yoga Studio,Sandwich Place,Beer Bar,Italian Restaurant,Fast Food Restaurant,Juice Bar,Seafood Restaurant
2,M5B,Downtown Toronto,"Ryerson ,Garden District",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bakery,Japanese Restaurant,Fast Food Restaurant,Plaza,Italian Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Restaurant,Italian Restaurant,Cocktail Bar,Bakery,Cosmetics Shop,Clothing Store,Hotel,Beer Bar
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Pub,Park,Neighborhood,Health Food Store,Trail,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop


Finally, separate neighborhoods into clusters.

In [86]:

address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [89]:
# create map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postcode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters